In [ ]:
from pymongo import MongoClient
import pandas as pd

In [ ]:
def connect_database(database_name : str) :
    """Create connexion to Mongo Database

    Args:
        database_name (str): Name of the database

    Returns:
        Client connexion
    """
    client = MongoClient()
    return client[database_name]

def get_collection(connection, collection_name : str) :
    """Connect to a Mongo collection

    Args:
        connection : Mongo client connection
        collection_name (str): Name of the collection

    Returns:
        _type_: The collection
    """
    return connection[collection_name]

In [ ]:
def read_and_create_dataframe(collection) -> pd.DataFrame :
    """Read a collection with League of Legends JSON data and create the associated dataframe
    Each row correspond to (match,participant) key.

    Args:
        collection (_type_): Mongo collection

    Returns:
        pd.DataFrame: DataFrame of the JSON data
    """
    df = pd.DataFrame()
    for game in collection.find() :
        df = pd.concat([df,pd.json_normalize(game)])
    df = df.explode('participants').reset_index(drop=True)
    df_participants = pd.json_normalize(df['participants'])
    df = pd.concat([df.drop(columns='participants'),df_participants],axis = 1)
    return df

In [ ]:
connect = connect_database('lol_match_database')
scrim_matches = get_collection(connect,"scrim_matches")
data_scrim_matches = read_and_create_dataframe(scrim_matches)
data_scrim_matches

In [ ]:
data_scrim_matches['PUUID']

### Team dico

In [ ]:
team_scald_dico = {
    "TOP" : ["62baa689-290d-5daa-be1a-a9da42fca824"],
    "JUNGLE" : ["90cb66b0-18fd-5689-bf52-c5a80c88cdc5"],
    "MIDDLE" : ["52334604-2f25-5556-8284-90370c43f6eb"],
    "BOT": ["9af56d81-a4c0-5447-b465-dd203dd80c6f"],
    "SUPPORT" : ["950ff368-e5a1-572e-84c1-824a6364dd3b"]
}

### 

In [ ]:
def filter_data_on_team(data : pd.DataFrame,team_dict : dict) -> pd.DataFrame :
    """Filter data based on a team PUUID dictionnary

    Args:
        data (pd.DataFrame): Input data
        team_dict (dict): Dictionnary containing PUUID. Example = {"TOP" : ["9df5d86"], "JUNGLE" : ...}

    Returns:
        pd.DataFrame: The filtered DataFrame
    """
    return data.loc[data['PUUID'].apply(lambda puuid: any(puuid in sublist for sublist in team_dict.values()))]

In [ ]:
scl = filter_data_on_team(data_scrim_matches,team_scald_dico)

In [ ]:
def get_winrate_by_side(data : pd.DataFrame) :
    """Compute winrate in blue and red side for a dataFrame (usage with team filter)

    Args:
        data (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    pass
    return #{"blue" : float(winrate_blue) , "red" : float(winrate_red)}

In [ ]:
get_winrate_by_side(scl)